In [1]:
import pandas as pd
import numpy as np

se importan el conjunto de datos de train/test

In [2]:
data_train = pd.read_csv('./data/application_train.csv')
data_test = pd.read_csv('./data/application_test.csv')

In [3]:
np.mean(data_train.TARGET==0), np.mean(data_train.TARGET==1)

(0.91927118054313506, 0.080728819456864956)

In [4]:
data_train_1 = data_train.loc[data_train.loc[data_train.index,'TARGET']==1]
data_train_0 = data_train.loc[data_train.loc[data_train.index,'TARGET']==0]

data_train_1.shape, data_train_0.shape

((24825, 122), (282686, 122))

Para balancear el dataset se recurre a elegir de las etiquetas 0, el mismo número de datos que las etiquetas 1 contienen

In [5]:
index_0 = np.random.choice(data_train_0.index,data_train_1.shape[0],False)

In [6]:
index_0

array([253907, 252950, 194469, ..., 105705, 239776,  85821])

In [7]:
data_train_0 = data_train_0.loc[index_0]
data_train_1.shape, data_train_0.shape

((24825, 122), (24825, 122))

In [8]:
data_train = pd.concat((data_train_0,data_train_1)).sort_index()
data_train.shape, data_test.shape

((49650, 122), (48744, 121))

In [9]:
y_train = data_train.TARGET
data_train = data_train[[col for col in data_train.columns if col!='TARGET']]

In [10]:
data_train = data_train.set_index(data_train.SK_ID_CURR)
data_test = data_test.set_index(data_test.SK_ID_CURR)

data_train = data_train.drop('SK_ID_CURR',axis=1)
data_test = data_test.drop('SK_ID_CURR',axis=1)

In [11]:
data_train.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100031,Cash loans,F,N,Y,0,112500.0,979992.0,27076.5,702000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0
100033,Cash loans,M,Y,Y,0,270000.0,790830.0,57676.5,675000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
100035,Cash loans,F,N,Y,0,292500.0,665892.0,24592.5,477000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,5.0
100037,Cash loans,F,N,N,0,90000.0,199008.0,20893.5,180000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


se considera que algunas columnas no aportan información siginificante, para esto se recurre a eliminarlas, dichas columnas se mensionan a continuación:

    -NAME_TYPE_SUITE: Quien acompañaba al cliente en el momento de solicitar el prestamo.
    -HOUR_APPR_PROCESS_START: hora aproximada en la que el cliente solicitó el prestamo.

In [12]:
data_train = data_train.drop(['NAME_TYPE_SUITE','HOUR_APPR_PROCESS_START'],axis=1)
data_test = data_test.drop(['NAME_TYPE_SUITE','HOUR_APPR_PROCESS_START'],axis=1)

así, por otro lado hay que decir que ciertas caracteristicas vienen representadas por valores los cuales están alejados de a realidad, por ejemplo las edades o dias, vienen de forma negativa, para esto se hará el respectivo cambio.

In [13]:
data_train.DAYS_BIRTH = abs(data_train.DAYS_BIRTH)/365 #años de las personas
data_test.DAYS_BIRTH = abs(data_test.DAYS_BIRTH)/365 #años de las personas

In [14]:
data_train.DAYS_EMPLOYED = abs(data_train.DAYS_EMPLOYED)
data_test.DAYS_EMPLOYED = abs(data_test.DAYS_EMPLOYED)

data_train.DAYS_REGISTRATION = abs(data_train.DAYS_REGISTRATION)
data_test.DAYS_REGISTRATION = abs(data_test.DAYS_REGISTRATION)

data_train.DAYS_LAST_PHONE_CHANGE = abs(data_train.DAYS_LAST_PHONE_CHANGE)
data_test.DAYS_LAST_PHONE_CHANGE = abs(data_test.DAYS_LAST_PHONE_CHANGE)

dado que la columa OWN_CAR_AGE puede ser significante ya que es a la edad del carro del cliente, para los clientes que no tienen vehiculo se procede a poner 0

In [15]:
data_train.OWN_CAR_AGE = data_train.OWN_CAR_AGE.fillna(value=0)
data_test.OWN_CAR_AGE = data_test.OWN_CAR_AGE.fillna(value=0)

ahora dadas las columnas de EXT_SOURCE_1,2 y 3 las cuales son una puntuación de base de datos esxternas, para los clientes que no tienen puntuación, se asumirá la media de la puntuación

In [18]:
data_train.EXT_SOURCE_1.min()

0.01469148240341735

In [19]:
data_train.EXT_SOURCE_1 = data_train.EXT_SOURCE_1.fillna(value=data_train.EXT_SOURCE_1.min())
data_test.EXT_SOURCE_1 = data_test.EXT_SOURCE_1.fillna(value=data_test.EXT_SOURCE_1.min())

data_train.EXT_SOURCE_2 = data_train.EXT_SOURCE_2.fillna(value=data_train.EXT_SOURCE_2.min())
data_test.EXT_SOURCE_2 = data_test.EXT_SOURCE_2.fillna(value=data_test.EXT_SOURCE_2.min())

data_train.EXT_SOURCE_3 = data_train.EXT_SOURCE_3.fillna(value=data_train.EXT_SOURCE_3.min())
data_test.EXT_SOURCE_3 = data_test.EXT_SOURCE_3.fillna(value=data_test.EXT_SOURCE_3.min())

se cargan las aplicaciones previas de aplicaciones

In [20]:
pre_app = pd.read_csv('./data/previous_application.csv')

In [21]:
pre_app = pre_app.set_index(pre_app.SK_ID_CURR.values)
pre_app = pre_app.drop('SK_ID_CURR',axis=1)

se codifican a valores númericos para poder ser utilizados

In [22]:
%run ./code/label_encoder.py
pre_app = label_encoder(pre_app)

debido a que un cliente puede tener varias aplicaciones previas, se toma la media de dichas aplicaciones para poder hacer el merge entre las tablas

In [23]:
pre_app = pre_app.groupby(pre_app.index).mean()

In [24]:
pre_app = pre_app.drop('SK_ID_PREV',axis=1)

In [25]:
data_train_merge = pd.merge(data_train, pre_app, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test, pre_app, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((49650, 153), (48744, 153))

ahora se procederá a cargar la tabla de bureau

In [26]:
bureau = pd.read_csv('./data/bureau.csv')
bureau.shape

(1716428, 17)

In [27]:
bureau = bureau.set_index(bureau.columns[0])
bureau = bureau.drop('SK_ID_BUREAU', axis=1)
bureau.head()

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,
215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [28]:
bureau = label_encoder(bureau)
bureau = bureau.groupby(bureau.index).mean()
bureau.shape

(305811, 15)

In [29]:
data_train_merge = pd.merge(data_train_merge, bureau, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test_merge, bureau, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((49650, 168), (48744, 168))

ahora se procede a cargar la tabla de credict card balance..

In [30]:
credict_cb = pd.read_csv('./data/credit_card_balance.csv')

In [31]:
credict_cb = credict_cb.set_index(credict_cb.columns[1])
credcit_cb = credict_cb.drop('SK_ID_PREV',axis=1)
print(credict_cb.shape)

(3840312, 22)


se codifica y se le saca la media ya que hay clientes con mas de un registro

In [32]:
credict_cb = label_encoder(credict_cb)
credict_cb = credict_cb.groupby(credict_cb.index).mean()
credict_cb.shape

(103558, 22)

se hace el merge con los datos anteriores...

In [33]:
data_train_merge = pd.merge(data_train_merge, credict_cb, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test_merge, credict_cb, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((49650, 190), (48744, 190))

ahora se procede a cargar el posh cash balance...

In [34]:
posh_cb = pd.read_csv('./data/POS_CASH_balance.csv')
posh_cb = posh_cb.set_index(posh_cb.columns[1])
posh_cb = posh_cb.drop('SK_ID_PREV',axis=1)
posh_cb.shape

(10001358, 6)

se codifica y se saca la media

In [35]:
posh_cb = label_encoder(posh_cb)
posh_cb = posh_cb.groupby(posh_cb.index).mean()
posh_cb.shape

(337252, 6)

se unen con los datos anteriores...

In [36]:
data_train_merge = pd.merge(data_train_merge, posh_cb, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test_merge, posh_cb, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((49650, 196), (48744, 196))

In [37]:
#lo mismo para el data_train
col_mean_train = (data_train_merge.loc[:,data_train_merge.count()<data_train_merge.shape[0]].min())
dict_col_mean_train = {}
for i in range(len(col_mean_train)):
    dict_col_mean_train[col_mean_train.index[i]] = col_mean_train[i]
    
#lo mismo para el data_test
col_mean_test = (data_test_merge.loc[:,data_test_merge.count()<data_test_merge.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [38]:
data_train_merge_t = data_train_merge.fillna(value=dict_col_mean_train)
data_test_merge_t = data_test_merge.fillna(value=dict_col_mean_test)

In [39]:
X_train = label_encoder(data_train_merge_t)
X_test = label_encoder(data_test_merge_t)

In [40]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [41]:
GBC.score(X_train, y_train)

0.71053373615307147

In [42]:
predict = GBC.predict_proba(X_test)[:,1]

In [43]:
predict

array([ 0.40042775,  0.6023168 ,  0.20172591, ...,  0.32304327,
        0.44176178,  0.73450376])

In [44]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=data_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_44.csv')